<a href="https://colab.research.google.com/github/Asato4931/Asato4931-CA_5_1-STS-Task-JSONL/blob/main/5_1_STS_Task_%2B_JSON_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from transformers import AutoTokenizer, AutoModelForMaskedLM

from sentence_transformers import SentenceTransformer
from sentence_transformers import models
 
import torch
import numpy as np

import pandas as pd
import json

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_selection import r_regression

from scipy import stats

# Load pre-trained tokenizer
transformer = models.Transformer("cl-tohoku/bert-base-japanese-v2")
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")
#model = AutoModelForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese-v2")
pooling = models.Pooling(transformer.get_word_embedding_dimension(), pooling_mode_mean_tokens=True, pooling_mode_cls_token=False, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[transformer, pooling])


valid_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/valid-v1.1.json', lines=True)


valid_data = valid_data.loc[:,['sentence1','sentence2']]

cos_sims = []
pearson = []
spearman = []



#メモ cos類似度の入力は、横ベクトルじゃないと食べてくれない


#Cos類似度

for i in range(0,1457):
  vector1_c = model.encode(valid_data.loc[i,'sentence1'])
  vector2_c = model.encode(valid_data.loc[i,'sentence2'])


  vector1_c = np.array(vector1_c).reshape(1,-1)
  vector2_c = np.array(vector2_c).reshape(1,-1)


  result = cosine_similarity(vector1_c,vector2_c)
  cos_sims.append(result)


#Pearson

for i in range(0,1457):
  vector1_p = model.encode(valid_data.loc[i,'sentence1'])
  vector2_p = model.encode(valid_data.loc[i,'sentence2'])

  result_p= stats.pearsonr(vector1_p,vector2_p)
  pearson.append(result_p.statistic)


#Spearman

for i in range(0,1457):
  vector1_s = model.encode(valid_data.loc[i,'sentence1'])
  vector2_s = model.encode(valid_data.loc[i,'sentence2'])

  vector1_s = np.array(vector1_s).reshape(1,-1)
  vector2_s = np.array(vector2_s).reshape(1,-1)
  result_s = stats.spearmanr(vector1_s,vector2_s ,axis=None)
  spearman.append(result_s.statistic)

pearson_df = pd.DataFrame(pearson)
pearson_df["metrics"] = "pearson"
pearson_df = pearson_df.rename(columns= { pearson_df.columns[0]: "score"})
pearson_df = pearson_df.reindex(columns=['metrics', 'score'])



spearman_df = pd.DataFrame(spearman)
spearman_df["metrics"] = "spearman"
spearman_df = spearman_df.rename(columns= { spearman_df.columns[0]: "score"})
spearman_df = spearman_df.reindex(columns=['metrics', 'score'])


frames = [pearson_df,spearman_df]

result_5_1 = pd.concat(frames)

result_5_1.to_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/CA_5_1 STS Task Results.jsonl', force_ascii=False, lines = True, orient='records' )


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install -U sentence-transformers

!apt-get install mecab mecab-ipadic-utf8 python-mecab libmecab-dev
!pip install mecab-python3 

!pip install fugashi

!pip install unidic_lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python-mecab
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658834 sha256=32d7f991c740f7705400e6ca24c800614104a246760ca6133dfe182bc621d393
  Stored in directory: /root/.cache/pip/wheels/56/9c/4f/2c115e896b4b6c584039ca19de3581d333856782ef108cdc5c
Successfully built unidic_lite
